In [1]:
""" 이 문서는 미국 배당주의 구매 시기 분석을 위한 데이터 수집 문서입니다.
해당 문서에 사용된 URL과 API는 다음과 같습니다.
주식 정보 크롤링 : 주식 별 주가 history = https://www.investing.com/, 주가, 배당금, 배당빈도 = https://www.stockanalysis.com"""

from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time
import csv
from datetime import datetime

bs = BeautifulSoup

**{키 : {딕셔너리}} 또는 {키 : 스트링}의 구조를 쉽게 만들기 위한 함수**

In [2]:
def make_list(name, item):
    """ 일반 주식인지 ETF인지의 구분이 필요하여 제작한 함수
    Args:
        name = 키값
        item = value 값 

    Returns:
        리스트 안에 딕셔너리 형식을 반환하며 상세 예제는 아래와 같습니다.
        name = 'stock'
        item = ['o','adm','aapl','msft']
        출력 -> [{'stocks': 'o'}, 
                {'stocks': 'adm'}, 
                {'stocks': 'aapl'}, 
                {'stocks': 'msft'}] 

    Raises:
        Error: item must be iterable value.
    """
    result = []
    for li in range(len(item)):
        result.append({name: item[li]})
    return result

**사이트명 리스트화를 위한 함수**

In [3]:
def stock_site(stocklist):
    """ 리스트 입력시 'stockanalysis.com' 사이트 내 dividend 항목으로 접속 가능한 주소를 일괄 정리해주는 함수
    Args:
        stock_type = 일반 'stock'인지, 'etf'인지 구분
        co_ticker = 주식 티커

    Returns:
        리스트 형식의 'stockanalysis.com'의 주소를 반환하며 상세 예제는 아래와 같습니다.
        stocklist = [{'stocks': 'o'}, 
                    {'stocks': 'adm'}, 
                    {'stocks': 'aapl'}, 
                    {'stocks': 'msft'}]
        출력 -> ['https://stockanalysis.com/stocks/o/dividend/',
                'https://stockanalysis.com/stocks/adm/dividend/',
                'https://stockanalysis.com/stocks/aapl/dividend/',
                'https://stockanalysis.com/stocks/msft/dividend/']
    """
    main_url = []
    for  stock_type, co_ticker in stocklist.items():
        main_url = [f"https://stockanalysis.com/{stock_type}/{co_ticker}/dividend/"]
    return main_url

**배당률 계산하기 쉽게 만든 함수**

In [5]:
def divi_rate(stock_price, dividend, divi_freq):
    """ 각 배당빈도 별 배당률을 기준으로 총 배당률을 계산합니다.
    배당률 X 배당빈도 / 현재가 X 100 
    Args:
        stock_price = 기준이 되는 주가
        dividend = 배당빈도별 배당금
        divi_freq = 구하고자 하는 배당빈도

    Returns:
        float 값을 반환합니다.
        stock_price = 60.99
        dividend = 0.2555
        divi_freq = 'Monthly'
        출력 -> 5.03
        """
    stock_price = float(stock_price)
    dividend = float(dividend)
    
    if divi_freq == 'Monthly':
        divi_freq = 12
    elif divi_freq == 'Quarterly':
        divi_freq = 4
    elif divi_freq == 'Semi-Annual':
        divi_freq = 2
    elif divi_freq == 'Annual':
        divi_freq = 1
    else:
        print('error')
        
    total_val = dividend * divi_freq / stock_price * 100 
    return f"{total_val:0.2f}"

**회사명 목록화**

In [6]:
""" 작업을 쉽게하기 위한 회사명 목록화

추출을 수월하게 하기 위해 일반 주식('stocks')과 ETF('etf')로 구분하여 딕셔너리 구조를 만든다.

이때 추출된 '현재가', '배당금', '배당빈도'을  딕셔너리 구조로 각 항목마다 포함시키기 위해
{딕셔너리{딕셔너리}} 구조로 만든다.
"""

stocks = ['o', 'adm', 'aapl', 'msft']
etf = ['jepi', 'spy']

company_name = make_list('stocks', stocks)
etf_name = make_list('etf',etf)

stock_list = company_name + etf_name

print(type(stock_list))
print(stock_list[:])

<class 'list'>
[{'stocks': 'o'}, {'stocks': 'adm'}, {'stocks': 'aapl'}, {'stocks': 'msft'}, {'etf': 'jepi'}, {'etf': 'spy'}]


**현재가, 배당금, 배당빈도 가져오기**

In [7]:
""" 본격적으로 데이터를 추출해보자

1. 접근이 막혀있어 헤더를 추가함 + requests.get은 지속적으로 403오류가 발생하여 Request로 접근함
2. URL 개수 만큼 루핑을 하기 위해 for문 적용함
3. 제일 먼저 각 항목별 URL을 추출함 

"""
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36'}

# URL 추출
url_list = []
for i in stock_list:
    for key in i.keys(): 
        url_list.extend(stock_site(i))

price_now = []
dividend = []
dividend_freq = []
for source in url_list:
    get_main_url = Request(source, headers=headers)
    main_html = urlopen(get_main_url).read()
    bs_main = bs(main_html, 'html.parser')

    # 각 종목별 현재가 추출
    price1 = bs_main.select_one("div[class='text-4xl font-bold inline-block']")
    price2 = bs_main.select_one("div[class='text-4xl font-bold block sm:inline']")

    if price1:
        all_price = price1
    elif price2:
        all_price = price2
    all_price = all_price.text
    price_now.append(all_price)
    
    # 각 종목별 배당금 추출
    divi_all = bs_main.select("td[class='svelte-1jtwn20']")[1].text
    divi_all = divi_all.replace('$','')
    dividend.append(divi_all)

    # 각 종목별 배당빈도 추출
    divi_year_all = bs_main.select("div[class='mt-0.5 text-lg font-semibold bp:text-xl sm:mt-1.5 sm:text-2xl']")[3].text
    dividend_freq.append(divi_year_all)

In [21]:
dividend

['0.2555', '0.450', '0.240', '0.680', '0.3593', '1.63837']

In [8]:
# 각 종목별 배당 테이블 데이터 내 배당금의 해당일 리스트 추출
table_dividend_list = []
for source in url_list:
    get_main_url = Request(source, headers=headers)
    main_html = urlopen(get_main_url).read()
    bs_main = bs(main_html, 'html.parser')

    table_date = bs_main.select("tbody >  tr[class=svelte-1jtwn20] > td:nth-child(4)")
    table_dividend = bs_main.select("tbody >  tr[class=svelte-1jtwn20] > td:nth-child(2)")
    
    table_dividend_date_list = []
    table_dividend_price_list = []
    for i in range(len(table_date)):
        table_dividend_price_1 = table_dividend[i].text # 과거 주가 추출
        table_dividend_price = float(table_dividend_price_1.replace("$", ""))
        table_dividend_date = table_date[i].text # 해당일 추출
        
        # May 15, 2023
        table_dividend_date = datetime.strptime(table_dividend_date, '%b %d, %Y')
        table_dividend_date_modify = table_dividend_date.strftime('%m/%d/%Y')
        if table_dividend_date >= datetime(2020, 1, 1):
            table_dividend_price_list.append(table_dividend_price)
            table_dividend_date_list.append(table_dividend_date_modify)

        # 최고 배당금의 해당하는 날짜를 추출하기 위한 딕셔너리 묶음화 {key - 날짜 : Value - 과거 주가} ->
        # 최고 배당금 수령 당시 주가를 확인하여 배당률을 확인하기 위함
        tabel_value_dict = dict(zip(table_dividend_date_list, table_dividend_price_list))
    table_dividend_list.append(tabel_value_dict)

In [9]:
# 모든 데이터 종합
all_info = []
stock_list_all = []
for sln in range(len(stock_list)):
    slv = list(stock_list[sln].values())
    stock_list_all.append(slv[0])
    all_info.append([(stock_list_all[sln]),
                     (price_now[sln]), 
                     (dividend_freq[sln]), 
                     (dividend[sln]), 
                     (divi_rate(price_now[sln],dividend[sln],dividend_freq[sln]))])

print(all_info)

[['o', '60.99', 'Monthly', '0.2555', '5.03'], ['adm', '80.75', 'Quarterly', '0.450', '2.23'], ['aapl', '193.61', 'Quarterly', '0.240', '0.50'], ['msft', '359.49', 'Quarterly', '0.680', '0.76'], ['jepi', '55.40', 'Monthly', '0.3593', '7.78'], ['spy', '454.19', 'Quarterly', '1.63837', '1.44']]


In [10]:
# conda install -c conda-forge selenium
# conda install -c conda-forge webdriver-manager
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager

**최고 배당율 알아보기**

In [11]:
""" INVESTING.COM 사이트를 통해 최고배당금이었을 때 당시 주가로 배당률을 알아보자.

stockanalysis.com이 아니라 investing.com으로 당시 주가 알아보는 이유:
stockanalysis의 과거 주가는 차트이나, investing.com는 과거 주가가 목록으로 정리되어 있어 과거 주가는 investin.com을 사용한다.

방법 1. 주가를 상징하는 티커는 동일하다.
방법 2. 티커를 기점으로 investing.com의 URL주소를 추출한다.
방법 3. 최고 배당금 당시 날짜를 찾아 해당일의 주가를 추출한다. (selenium 사용)
방법 4. 만들어둔 함수를 통해 배당률을 구한다. (divi_rate)

* selenium 사용 시
  > URL 주소 추출은 Request를 사용함(오류나서 안쓰는 것으로 변경)
  """

# 티커를 기준으로 검색 시 제일 위에 있는 항목의 주소를 가져오도록 요청.
investing_com_urls = []
for iu in all_info:
  inv_co_name = iu[0]
  inv_main_url = Request(f"https://www.investing.com/search/?q={inv_co_name}", headers=headers) # investing.com은 티커를 URL에 사용하지 않아 직접 검색 후 찾아야함
  inv_html = urlopen(inv_main_url).read()
  inv_main = bs(inv_html, 'html.parser')
  inv_stock = inv_main.select_one("span[class='second']") # 해당되는 티커의 태그를 반환함
  investing_com_urls.append(f"https://www.investing.com{inv_stock.find_parent('a').get('href')}-historical-data") # 티커 태그의 부모 태그 중 URL 주소가 담긴 href 요소를 추출함

print(investing_com_urls)


['https://www.investing.com/equities/realty-income-historical-data', 'https://www.investing.com/equities/archer-daniels-mid-historical-data', 'https://www.investing.com/equities/apple-computer-inc-historical-data', 'https://www.investing.com/equities/microsoft-corp-historical-data', 'https://www.investing.com/etfs/jepi-historical-data', 'https://www.investing.com/etfs/spdr-s-p-500-historical-data']


**최고 배당율 알아보기2 (selenuim)**

In [13]:
# investing.com 사이트 내 각 종목별 3년치 종가 추출
options = Options()
options.add_argument("--headless") # 브라우저 안뜨게 하는 옵션 (작업 시간 단축)
options.add_argument("--window-size= 968, 1012")
options.add_argument('--blink-settings=imagesEnabled=false') # 이미지파일 로딩 제한으로 속도 향상
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)\
                      AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36')

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

dfe_dict_list = []
for ius in range(len(investing_com_urls)):
    iusi = investing_com_urls[ius]
    driver.get(iusi)

    driver.implicitly_wait(10)
    driver.find_element(By.CSS_SELECTOR, ".DatePickerWrapper_center__BYe4Q").click()
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, ".HistoryDatePicker_HistoryDatePicker__IOT1L > div:nth-child(1) > div > div:nth-child(1)").click()
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, ".HistoryDatePicker_HistoryDatePicker__IOT1L > div:nth-child(1) > div > div:nth-child(1) > input[type=date]").clear()
    driver.find_element(By.CSS_SELECTOR, ".HistoryDatePicker_HistoryDatePicker__IOT1L > div:nth-child(1) > div > div:nth-child(1) > input[type=date]").send_keys("2020-01-01")
    driver.find_element(By.CSS_SELECTOR, ".HistoryDatePicker_HistoryDatePicker__IOT1L > div.HistoryDatePicker_footer__xzpr0 > button").click()
    
    time.sleep(10)
    
    dfe_price = driver.find_elements(By.XPATH,"//main//tbody//td[contains(@dir,'ltr') and not(contains(text(),'%'))]")
    dfe_date = driver.find_elements(By.XPATH,"//main//tbody[contains(@class, 'datatable_body__tb4jX')]//td[contains(@class, 'datatable_cell__LJp3C font-bold')]/time")

    ep_list = [dfep_v.text for dfep_v in dfe_price]
    ed_list = [dfed_v.text for dfed_v in dfe_date]

    dfe_dict = dict(zip(ed_list, ep_list))
    dfe_dict_list.append(dfe_dict)

driver.close()
driver.quit()

In [14]:
# 배당금 일자와 종가별 일자를 대입해 같은 날짜면 해당 배당금과 종가를 계산하여 배당률 계산
highest_dividend_list = []
for list in range(len(stock_list)):
    highest_dividend_dict = {}
    for key, value in table_dividend_list[list].items():
        if  key in dfe_dict_list[list].keys():
            highest_dividend_rate = divi_rate(dfe_dict_list[list][key], 
                                                table_dividend_list[list][key],
                                                dividend_freq[list])
            highest_dividend_dict[key] = highest_dividend_rate
    highest_dividend_list.append(highest_dividend_dict)

In [18]:
# 현재 배당률과 3년 내 최고 배당률을 비교하여 투자할지 말지 결정하기 위한 표 산출
stock_cols = ['티커', '현재가($)', '배당빈도', '배당금($)', '현재 배당률(%)', '3년 내 최고 배당률(%)']

highest_dividend = []
for i in range(len(stock_list)):
    highest_dividend.append(max(highest_dividend_list[i].values()))

all_info_final = np.c_[all_info, highest_dividend]

stock_df = pd.DataFrame(all_info_final, columns=stock_cols)
stock_df.to_csv('sujong_project_01', index='false')

print("기준 날짜 : ", datetime.now().date())
stock_df

기준 날짜 :  2023-07-19


,티커,현재가($),배당빈도,배당금($),현재 배당률(%),3년 내 최고 배당률(%)
0,o,60.99,Monthly,0.2555,5.03,5.42
1,adm,80.75,Quarterly,0.450,2.23,3.76
2,aapl,193.61,Quarterly,0.240,0.50,1.06
3,msft,359.49,Quarterly,0.680,0.76,1.47
4,jepi,55.40,Monthly,0.3593,7.78,9.86
5,spy,454.19,Quarterly,1.63837,1.44,1.95


### 배당금 보는 방법

1. 현재 배당률이 3년 내 최고 배당률에 근접한 종목 대상 
   (개인적으로 0.2~0.5% 내 들어온 종목을 보지만, 섹터별, 전문가별 매매 기준이 다름으로 본인이 판단했을 때 감당 가능한 기준을 정해놓는게 좋다.)
2. 재무재표 상 순이익(net income)이 전년도 대비 -50% 이상이며, 자본 잠식 상태가 아닐 경우를 기준으로 삼는다.
3. 회사명을 검색했을 떄 부정적인 뉴스나 이슈가 별도로 있는지 확인한다.
4. 환율은 1분기 내 10% 이상 급격하게 오르는 상황이 아니면 신경쓰지 않는다. 